# DeepSeek API 多轮对话 - OpenAI SDK

> 指导文章：[DeepSeek API 多轮对话 - OpenAI SDK](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/DeepSeek%20API%20多轮对话%20-%20OpenAI%20SDK.md)
> 
> 在代码运行过程中可能遇到的问题：
> 1. 为什么等待时间那么长？
>
>    答：因为默认参数 `stream=False`，不启用流式输出。对于推理模型来说，非流式输出下的用户体验极其糟糕，故当前代码文件为最后一次非流式输出的代码演示，之后将默认采用流式输出。
> 2. 504 报错：对应服务当前的负载高，可以稍后尝试或者切换平台。

从下方选择平台开始，替换 `your-api-key` 后点击 `►` 或使用 `Shift + 回车` 运行代码块。

**注意**：

- 当前代码为非流式输出。

在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/deepseek-api-guide-4) | [Colab](https://colab.research.google.com/drive/11REpPevN4bSL8yeJzQs7LAiV2cm9lUmh?usp=sharing)

# DeepSeek

## DeepSeek-Chat

In [ ]:
from openai import OpenAI
import os

# 临时环境变量配置
os.environ["DEEPSEEK_API_KEY"] = "your-api-key"

client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com",
)

### 单轮对话

In [ ]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 打印模型消息
print(completion.choices[0].message.content)

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

# 将第一轮模型回复加入历史记录
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    # 获取用户输入
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break
        
    # 记录用户输入
    messages.append({'role': 'user', 'content': user_input})
    
    # 调用 API 获取模型回复
    completion = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    content = completion.choices[0].message.content
    print("AI:", content)
    
    # 记录模型回复
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ChatSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - content (str)：模型的回复内容
        """
        self.append_message('user', user_input)
        completion = self.client.chat.completions.create(
            model="deepseek-chat",
            messages=self.messages
        )
        content = completion.choices[0].message.content
        self.append_message('assistant', content)
        return content


# 初始化对话会话，采用默认的系统消息
session = ChatSession(client)

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    reply = session.get_response(user_input)
    print("AI:", reply)

## DeepSeek-Reasoner

### 单轮对话

In [ ]:
completion = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 模型推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\n{content}\n")

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第一轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

# 只将最终的回复（content）加入对话历史
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第二轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:
        break

    # 1. 记录用户输入
    messages.append({'role': 'user', 'content': user_input})

    # 2. 调用 API
    completion = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=messages
    )
    reasoning_content = completion.choices[0].message.reasoning_content
    content = completion.choices[0].message.content

    # 3. 输出推理思考过程与最终回复
    print(f"===== 推理过程 =====\n{reasoning_content}\n")
    print(f"===== 模型回复 =====\nAI: {content}\n")

    # 4. 只将模型的最终回复写入到历史消息中
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ReasonerSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取思维链和回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - (reasoning_content, content) (tuple)：模型的推理过程和回复
        """
        # 1. 记录用户输入
        self.append_message('user', user_input)

        # 2. 调用 API
        completion = self.client.chat.completions.create(
            model="deepseek-reasoner",
            messages=self.messages
        )

        # 3. 获取推理思考过程和最终回复
        reasoning_content = completion.choices[0].message.reasoning_content
        content = completion.choices[0].message.content

        # 4. 只将模型的最终回复加入到历史消息
        self.append_message('assistant', content)

        return reasoning_content, content


# 初始化推理会话，采用默认的系统消息
session = ReasonerSession(client)

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break

    reasoning, reply = session.get_response(user_input)
    print(f"===== 推理过程 =====\n{reasoning}\n")
    print(f"===== 模型回复 =====\nAI: {reply}\n")

# 硅基流动

## DeepSeek-Chat

In [ ]:
from openai import OpenAI
import os

# 临时环境变量配置
os.environ["SILICONFLOW_API_KEY"] = "your-api-key"

client = OpenAI(
    api_key=os.getenv("SILICONFLOW_API_KEY"),
    base_url="https://api.siliconflow.cn/v1",
)

### 单轮对话

In [ ]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 打印模型消息
print(completion.choices[0].message.content)

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

# 将第一轮模型回复加入历史记录
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    # 获取用户输入
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break
        
    # 记录用户输入
    messages.append({'role': 'user', 'content': user_input})
    
    # 调用 API 获取模型回复
    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3",
        messages=messages
    )
    content = completion.choices[0].message.content
    print("AI:", content)
    
    # 记录模型回复
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ChatSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - content (str)：模型的回复内容
        """
        self.append_message('user', user_input)
        completion = self.client.chat.completions.create(
            model="deepseek-ai/DeepSeek-V3",
            messages=self.messages
        )
        content = completion.choices[0].message.content
        self.append_message('assistant', content)
        return content


# 初始化对话会话，采用默认的系统消息
session = ChatSession(client)

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    reply = session.get_response(user_input)
    print("AI:", reply)

## DeepSeek-Reasoner

### 单轮对话

In [ ]:
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 模型推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\n{content}\n")

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第一轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

# 只将最终的回复（content）加入对话历史
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第二轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:
        break

    # 1. 记录用户输入
    messages.append({'role': 'user', 'content': user_input})

    # 2. 调用 API
    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1",
        messages=messages
    )
    reasoning_content = completion.choices[0].message.reasoning_content
    content = completion.choices[0].message.content

    # 3. 输出推理思考过程与最终回复
    print(f"===== 推理过程 =====\n{reasoning_content}\n")
    print(f"===== 模型回复 =====\nAI: {content}\n")

    # 4. 只将模型的最终回复写入到历史消息中
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ReasonerSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取思维链和回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - (reasoning_content, content) (tuple)：模型的推理过程和回复
        """
        # 1. 记录用户输入
        self.append_message('user', user_input)

        # 2. 调用 API
        completion = self.client.chat.completions.create(
            model="deepseek-ai/DeepSeek-R1",
            messages=self.messages
        )

        # 3. 获取推理思考过程和最终回复
        reasoning_content = completion.choices[0].message.reasoning_content
        content = completion.choices[0].message.content

        # 4. 只将模型的最终回复加入到历史消息
        self.append_message('assistant', content)

        return reasoning_content, content


# 初始化推理会话，采用默认的系统消息
session = ReasonerSession(client)

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break

    reasoning, reply = session.get_response(user_input)
    print(f"===== 推理过程 =====\n{reasoning}\n")
    print(f"===== 模型回复 =====\nAI: {reply}\n")

# 阿里云百炼

## DeepSeek-Chat

In [ ]:
from openai import OpenAI
import os

# 临时环境变量配置
os.environ["DASHSCOPE_API_KEY"] = "your-api-key"

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

### 单轮对话

In [ ]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-v3",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 打印模型消息
print(completion.choices[0].message.content)

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-v3",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

# 将第一轮模型回复加入历史记录
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-v3",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    # 获取用户输入
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break
        
    # 记录用户输入
    messages.append({'role': 'user', 'content': user_input})
    
    # 调用 API 获取模型回复
    completion = client.chat.completions.create(
        model="deepseek-v3",
        messages=messages
    )
    content = completion.choices[0].message.content
    print("AI:", content)
    
    # 记录模型回复
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ChatSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - content (str)：模型的回复内容
        """
        self.append_message('user', user_input)
        completion = self.client.chat.completions.create(
            model="deepseek-v3",
            messages=self.messages
        )
        content = completion.choices[0].message.content
        self.append_message('assistant', content)
        return content


# 初始化对话会话，采用默认的系统消息
session = ChatSession(client)

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    reply = session.get_response(user_input)
    print("AI:", reply)

## DeepSeek-Reasoner

### 单轮对话

In [ ]:
completion = client.chat.completions.create(
    model="deepseek-r1",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 模型推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\n{content}\n")

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-r1",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第一轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

# 只将最终的回复（content）加入对话历史
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-r1",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第二轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:
        break

    # 1. 记录用户输入
    messages.append({'role': 'user', 'content': user_input})

    # 2. 调用 API
    completion = client.chat.completions.create(
        model="deepseek-r1",
        messages=messages
    )
    reasoning_content = completion.choices[0].message.reasoning_content
    content = completion.choices[0].message.content

    # 3. 输出推理思考过程与最终回复
    print(f"===== 推理过程 =====\n{reasoning_content}\n")
    print(f"===== 模型回复 =====\nAI: {content}\n")

    # 4. 只将模型的最终回复写入到历史消息中
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ReasonerSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取思维链和回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - (reasoning_content, content) (tuple)：模型的推理过程和回复
        """
        # 1. 记录用户输入
        self.append_message('user', user_input)

        # 2. 调用 API
        completion = self.client.chat.completions.create(
            model="deepseek-r1",
            messages=self.messages
        )

        # 3. 获取推理思考过程和最终回复
        reasoning_content = completion.choices[0].message.reasoning_content
        content = completion.choices[0].message.content

        # 4. 只将模型的最终回复加入到历史消息
        self.append_message('assistant', content)

        return reasoning_content, content


# 初始化推理会话，采用默认的系统消息
session = ReasonerSession(client)

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break

    reasoning, reply = session.get_response(user_input)
    print(f"===== 推理过程 =====\n{reasoning}\n")
    print(f"===== AI回复 =====\nAI: {reply}\n")

# 百度智能云

## DeepSeek-Chat

In [ ]:
from openai import OpenAI
import os

# 临时环境变量配置
os.environ["BAIDU_API_KEY"] = "your-api-key"

client = OpenAI(
    api_key=os.getenv("BAIDU_API_KEY"),
    base_url="https://qianfan.baidubce.com/v2",
)

### 单轮对话

In [ ]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-v3",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 打印模型消息
print(completion.choices[0].message.content)

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-v3",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

# 将第一轮模型回复加入历史记录
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-v3",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    # 获取用户输入
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break
        
    # 记录用户输入
    messages.append({'role': 'user', 'content': user_input})
    
    # 调用 API 获取模型回复
    completion = client.chat.completions.create(
        model="deepseek-v3",
        messages=messages
    )
    content = completion.choices[0].message.content
    print("AI:", content)
    
    # 记录模型回复
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ChatSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - content (str)：模型的回复内容
        """
        self.append_message('user', user_input)
        completion = self.client.chat.completions.create(
            model="deepseek-v3",
            messages=self.messages
        )
        content = completion.choices[0].message.content
        self.append_message('assistant', content)
        return content


# 初始化对话会话，采用默认的系统消息
session = ChatSession(client)

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    reply = session.get_response(user_input)
    print("AI:", reply)

## DeepSeek-Reasoner

### 单轮对话

In [ ]:
completion = client.chat.completions.create(
    model="deepseek-r1",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 模型推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\n{content}\n")

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-r1",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第一轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

# 只将最终的回复（content）加入对话历史
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-r1",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第二轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:
        break

    # 1. 记录用户输入
    messages.append({'role': 'user', 'content': user_input})

    # 2. 调用 API
    completion = client.chat.completions.create(
        model="deepseek-r1",
        messages=messages
    )
    reasoning_content = completion.choices[0].message.reasoning_content
    content = completion.choices[0].message.content

    # 3. 输出推理思考过程与最终回复
    print(f"===== 推理过程 =====\n{reasoning_content}\n")
    print(f"===== 模型回复 =====\nAI: {content}\n")

    # 4. 只将模型的最终回复写入到历史消息中
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ReasonerSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取思维链和回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - (reasoning_content, content) (tuple)：模型的推理过程和回复
        """
        # 1. 记录用户输入
        self.append_message('user', user_input)

        # 2. 调用 API
        completion = self.client.chat.completions.create(
            model="deepseek-r1",
            messages=self.messages
        )

        # 3. 获取推理思考过程和最终回复
        reasoning_content = completion.choices[0].message.reasoning_content
        content = completion.choices[0].message.content

        # 4. 只将模型的最终回复加入到历史消息
        self.append_message('assistant', content)

        return reasoning_content, content


# 初始化推理会话，采用默认的系统消息
session = ReasonerSession(client)

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break

    reasoning, reply = session.get_response(user_input)
    print(f"===== 推理过程 =====\n{reasoning}\n")
    print(f"===== AI回复 =====\nAI: {reply}\n")

# 字节火山引擎

## DeepSeek-Chat

In [ ]:
from openai import OpenAI
import os

# 临时环境变量配置
os.environ["ARK_API_KEY"] = "your-api-key"

client = OpenAI(
    api_key=os.getenv("ARK_API_KEY"),
    base_url="https://ark.cn-beijing.volces.com/api/v3",
)

### 单轮对话

In [ ]:
# 单轮对话示例
completion = client.chat.completions.create(
    model="deepseek-v3-241226",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 打印模型消息
print(completion.choices[0].message.content)

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-v3-241226",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

# 将第一轮模型回复加入历史记录
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-v3-241226",
    messages=messages
)
content = completion.choices[0].message.content
print("AI:", content)

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    # 获取用户输入
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break
        
    # 记录用户输入
    messages.append({'role': 'user', 'content': user_input})
    
    # 调用 API 获取模型回复
    completion = client.chat.completions.create(
        model="deepseek-v3-241226",
        messages=messages
    )
    content = completion.choices[0].message.content
    print("AI:", content)
    
    # 记录模型回复
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ChatSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - content (str)：模型的回复内容
        """
        self.append_message('user', user_input)
        completion = self.client.chat.completions.create(
            model="deepseek-v3-241226",
            messages=self.messages
        )
        content = completion.choices[0].message.content
        self.append_message('assistant', content)
        return content


# 初始化对话会话，采用默认的系统消息
session = ChatSession(client)

print("开始对话，输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    reply = session.get_response(user_input)
    print("AI:", reply)

## DeepSeek-Reasoner

### 单轮对话

In [ ]:
completion = client.chat.completions.create(
    model="deepseek-r1-250120",
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {'role': 'user', 'content': '你是谁？'}
    ]
)

# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 模型推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\n{content}\n")

### 两轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 第一轮对话
messages.append({'role': 'user', 'content': '9.11 和 9.8 哪个更大？'})
completion = client.chat.completions.create(
    model="deepseek-r1-250120",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第一轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

# 只将最终的回复（content）加入对话历史
messages.append({'role': 'assistant', 'content': content})

# 第二轮对话
messages.append({'role': 'user', 'content': '刚刚针对哪两个数值进行了比较？'})
completion = client.chat.completions.create(
    model="deepseek-r1-250120",
    messages=messages
)
# 获取推理思考过程和模型回复
reasoning_content = completion.choices[0].message.reasoning_content
content = completion.choices[0].message.content

print(f"===== 第二轮推理过程 =====\n{reasoning_content}\n")
print(f"===== 模型回复 =====\nAI: {content}\n")

### 多轮对话

In [ ]:
# 初始化对话历史，包含系统消息
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:
        break

    # 1. 记录用户输入
    messages.append({'role': 'user', 'content': user_input})

    # 2. 调用 API
    completion = client.chat.completions.create(
        model="deepseek-r1-250120",
        messages=messages
    )
    reasoning_content = completion.choices[0].message.reasoning_content
    content = completion.choices[0].message.content

    # 3. 输出推理思考过程与最终回复
    print(f"===== 推理过程 =====\n{reasoning_content}\n")
    print(f"===== 模型回复 =====\nAI: {content}\n")

    # 4. 只将模型的最终回复写入到历史消息中
    messages.append({'role': 'assistant', 'content': content})

### 封装对话逻辑

In [ ]:
class ReasonerSession:
    def __init__(self, client, system_message="You are a helpful assistant."):
        """
        参数：
        - client (openai.OpenAI): OpenAI 客户端实例
        - system_message (str): 系统消息，用于设定对话背景
        """
        self.client = client
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息
        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'。
        - content (str): 消息内容。
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input):
        """
        添加用户消息，调用 API 获取思维链和回复，并将回复加入历史记录
        参数：
        - user_input (str): 用户输入的消息
        返回：
        - (reasoning_content, content) (tuple)：模型的推理过程和回复
        """
        # 1. 记录用户输入
        self.append_message('user', user_input)

        # 2. 调用 API
        completion = self.client.chat.completions.create(
            model="deepseek-r1-250120",
            messages=self.messages
        )

        # 3. 获取推理思考过程和最终回复
        reasoning_content = completion.choices[0].message.reasoning_content
        content = completion.choices[0].message.content

        # 4. 只将模型的最终回复加入到历史消息
        self.append_message('assistant', content)

        return reasoning_content, content


# 初始化推理会话，采用默认的系统消息
session = ReasonerSession(client)

print("开始对话（Reasoner），输入 'exit' 或 'quit' 结束对话。")
while True:
    user_input = input("You：")
    if user_input.lower() in ['exit', 'quit']:  # 退出条件
        break

    reasoning, reply = session.get_response(user_input)
    print(f"===== 推理过程 =====\n{reasoning}\n")
    print(f"===== AI回复 =====\nAI: {reply}\n")